# I. Tweet Data Cleaning



In [1]:
## Tweet Table

In [1]:
import pymongo
from pymongo import MongoClient
from pprint import pprint
import pandas as pd
import json
import time
import re
import datetime
pd.set_option('display.max_rows', 5)
pd.set_option('display.max_colwidth', -1)

In [2]:
# function to cheack whether an object (data) contains attribute (x) or not
def check_var(data, x):
    try:
        data[x]
        return True
    except:
        return False

In [3]:
%%time
# making mongoDB database and collection
client = MongoClient('localhost', 27017)
db = client['mydatabase']

want_drop = input("Do you want to drop collection (yes/no): ")
if(want_drop == 'yes'): ## if type yes, then it will drop tweet table from mongodb to prepare new importing.
    db.tweet.drop()
    
tweet = db['tweet']

Do you want to drop collection (yes/no): no
Wall time: 1.48 s


In [5]:
#Import the necessary methods from tweepy library
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream

In [6]:
# yelin directory '/Users/yelin/Downloads/corona-out-3/corona-out-3'
# Kanya 'C:/Users/maxmo/OneDrive/Desktop/Rutgers/Spring2020/Data Management/corona-out-3'
# Hui Wang  file = '/Users/huiwang/Downloads/corona-out-3'
file = '/Users/huiwang/Downloads/corona-out-3'

In [7]:
%%time
# id, user_id, created_at, text, retweet_count, favorite_count, reply_count, hashtags
import json
with open(file, 'r', encoding = 'utf8') as f1 :
    for line in f1 :
        try :
            data = json.loads(line)
        except :
            continue
        
        hashtag = [] #to collect list of hashtags
        
        if (data['lang'] == 'en') :
            
            # case 1: check whether it is a retweet data or not
            if (check_var(data, 'retweeted_status')) :

                Created_at = pd.to_datetime(data['retweeted_status']['created_at'])
                UserID = data['retweeted_status']['user']['id']
                ID = data['retweeted_status']['id']
                retweet_count = data['retweeted_status']['retweet_count']
                favorite_count = data['retweeted_status']['favorite_count']
                reply_count = data['retweeted_status']['reply_count']

                # check whether there is extension version of that tweet or not        
                if (check_var(data['retweeted_status'], 'extended_tweet')) :
                    Text = data['retweeted_status']['extended_tweet']['full_text']
                    for t in data['retweeted_status']['extended_tweet']['entities']['hashtags'] :
                        hashtag.append(t['text'])                
                else :
                    Text = data['retweeted_status']['text']
                    for t in data['retweeted_status']['entities']['hashtags'] :
                        hashtag.append(t['text'])

                # check whether it is retweet of quoted tweet or not
                if (check_var(data['retweeted_status'], 'quoted_status')) :

                    #check whether there is extension version of the original tweet or not 
                    if (check_var(data['retweeted_status']['quoted_status'], 'extended_tweet')) :
                        Text = Text + ' || ' + data['retweeted_status']['quoted_status']['extended_tweet']['full_text']
                        for t in data['retweeted_status']['quoted_status']['extended_tweet']['entities']['hashtags'] :
                            hashtag.append(t['text'])                
                    else :
                        Text = Text + ' || ' + data['retweeted_status']['quoted_status']['text']
                        for t in data['retweeted_status']['quoted_status']['entities']['hashtags'] :
                            hashtag.append(t['text'])               
 
            # case 2: check whether it is a quote tweet data or not
            elif (check_var(data, 'quoted_status')) :
                Created_at = pd.to_datetime(data['created_at'])
                UserID = data['user']['id']
                ID = data['id']
                retweet_count = data['retweet_count']
                favorite_count = data['favorite_count']
                reply_count = data['reply_count']

                # check whether there is extension version of that quote tweet or not 
                if(check_var(data, 'extended_tweet')) :
                    Text = data['extended_tweet']['full_text'] + ' || ' 
                else:
                    Text = data['text'] + ' || '

                for t in data['entities']['hashtags'] :
                    hashtag.append(t['text'])

                # check whether there is extension version of the originl tweet or not 
                if (check_var(data['quoted_status'], 'extended_tweet')) :
                    Text = Text + data['quoted_status']['extended_tweet']['full_text']
                    for t in data['quoted_status']['extended_tweet']['entities']['hashtags'] :
                        hashtag.append(t['text'])                
                else:
                    Text = Text + data['quoted_status']['text']
                    for t in data['quoted_status']['entities']['hashtags'] :
                        hashtag.append(t['text'])

            # if it does not belong to case 1 and case 2, it is an ordinary tweet data
            else :
                Created_at = pd.to_datetime(data['created_at'])
                UserID = data['user']['id']
                ID = data['id']
                retweet_count = data['retweet_count']
                favorite_count = data['favorite_count']
                reply_count = data['reply_count']

                # check whether there is extension version of that tweet or not  
                if (check_var(data, 'extended_tweet')) :
                    Text = data['extended_tweet']['full_text']
                else :
                    Text = data['text']

                for t in data['entities']['hashtags'] :
                    hashtag.append(t['text'])

            tw = {'id' : ID,
                  'user_id' : UserID,
                  'created_at' : Created_at,
                  'text' : Text,
                  'retweet_count' : retweet_count,
                  'favorite_count' : favorite_count,
                  'reply_count' : reply_count,
                  'hashtags': hashtag
                 }

            # if the data is already in our database, we update it 
            if (tweet.count_documents({'id' : tw['id']}) > 0) :
                db.tweet.update_one({'id' : tw['id']},
                                    {'$set' : {'retweet_count' : tw['retweet_count'],
                                               'favorite_count' : tw['favorite_count'],
                                               'reply_count' : tw['reply_count']}})
            # if it is not, we insert it    
            else :
                tweet.insert_one(tw)

CPU times: user 1min 37s, sys: 5.99 s, total: 1min 43s
Wall time: 29min 47s


'id_1'

In [6]:
# create indexes for tweet id, text, hashtags and created_at
tweet.create_index([ ('id', 1) ])
tweet.create_index([ ('text', 1) ])
tweet.create_index([ ('hashtags', 1) ])
tweet.create_index([ ('created_at', 1) ])

'created_at_1'

In [4]:
tweet_table = pd.DataFrame(tweet.find())
tweet_table

,_id,id,user_id,created_at,text,retweet_count,favorite_count,reply_count,hashtags
0,5ea8a2f81d0c0c01823fe49a,1253949413191344128,207809313,2020-04-25 07:30:12,India’s war with Corona is ongoing.\n\nPlay your part and make sure no one around you starves.\n\nYour contribution to defeat hunger is as valuable as defeating Corona.\n\n#FeedTheNeedy https://t.co/TUWf2nqPgZ,398,2144,108,[FeedTheNeedy]
1,5ea8a2f81d0c0c01823fe49b,1253992905703862272,858859031464751104,2020-04-25 10:23:01,VIDEO | 25.04.20\n\n&gt; Monsta X appears on Yonhap News TV - Idol comeback through Corona... Turning on the Music industry.\n\n🔗https://t.co/kW5hsP5DeD\n\n#몬스타엑스 #MONSTAX\n#판타지아엑스 #FANTASIA_X\n@OfficialMonstaX\n\nhttps://t.co/tJN4Iyv1Gh https://t.co/GukJXWjaPW,59,144,1,"[몬스타엑스, MONSTAX, 판타지아엑스, FANTASIA_X]"
...,...,...,...,...,...,...,...,...,...
19074,5ea8a4261d0c0c0182402f1c,1254059747738816512,3130596629,2020-04-25 14:48:37,We have 1/4 of all deaths from #Corona - thanks to our 3rd world response by @realDonaldTrump #ashamed \n@Arpx #arleg https://t.co/ELyI69q8pp,0,0,0,"[Corona, ashamed]"
19075,5ea8a4261d0c0c0182402f1d,1254059751379361793,2279976427,2020-04-25 14:48:38,@VSReddy_MP I think wantedly you ate damaging your leaders image @ysjagan One side there is an increase of Corona cases and also @srikakulamdistrict now attacked with corona on the other side you are giving these tweets How to understand it @ArogyaAndhra @AndhraPradeshCM @apfightscorona,0,0,0,[]






# II. User Data Cleaning





In [11]:
## User Table

In [30]:
import pandas as pd
import psycopg2
import json

In [13]:
## Creating the database
conn = psycopg2.connect("dbname=postgres port=5432 user=postgres password=password")
conn.set_session(autocommit = True)
cur = conn.cursor()

cur.execute("CREATE DATABASE twitter_users" )
conn.commit()
cur.close()
conn.close()

DuplicateDatabase: database "twitter_users" already exists


In [14]:
# Creating the table
conn = psycopg2.connect("dbname=twitter_users port=5432 user=postgres password=password")
cur = conn.cursor()
#cur.execute("drop table users_data")
cur.execute("CREATE TABLE IF NOT EXISTS users_data( user_id bigint PRIMARY KEY, name varchar(255), user_name varchar(255), verified_status boolean,followers_count integer,friends_count integer,statuses_count integer,user_location varchar(500),favourites_count integer);" )
conn.commit()
cur.close()
conn.close()

In [15]:
def check_var(data,x):
    try:
        data[x]
        return True
    except:
        return False

In [28]:
%%time
# This function takes the the variable inputs and stores it into a PostgreSQL database
def store_data(user_id, name, user_name, verified_status, followers_count, friends_count, statuses_count, user_location, favourites_count):
    conn = psycopg2.connect("dbname=twitter_users port=5432 user=postgres password=password")
    cur = conn.cursor()
    insert_query = "INSERT INTO users_data (user_id, name, user_name, verified_status, followers_count, friends_count, statuses_count, user_location, favourites_count) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s) ON CONFLICT (user_id) DO UPDATE SET name=EXCLUDED.name, user_name=EXCLUDED.user_name, verified_status=EXCLUDED.verified_status, followers_count=EXCLUDED.followers_count, friends_count=EXCLUDED.friends_count, statuses_count=EXCLUDED.statuses_count, user_location=EXCLUDED.user_location, favourites_count=EXCLUDED.favourites_count"
    cur.execute(insert_query, (user_id, name, user_name, verified_status, followers_count, friends_count, statuses_count, user_location,favourites_count))
    conn.commit()
    cur.close()
    conn.close()
    
    
# Decoding the JSON file
with open(file, 'r', encoding = 'utf8') as f1:
    for line in f1:
        
        try:
            data = json.loads(line)
        except:
            continue
        
        if (data['lang'] == 'en'): # only 
            
            if (check_var(data, 'retweeted_status')):

                user_id = data['retweeted_status']['user']['id']
                name = data['retweeted_status']['user']['name']
                user_name = data['retweeted_status']['user']['screen_name']
                verified_status = data['retweeted_status']['user']['verified']
                followers_count = data['retweeted_status']['user']['followers_count']
                friends_count = data['retweeted_status']['user']['friends_count']
                statuses_count = data['retweeted_status']['user']['statuses_count']
                user_location = data['retweeted_status']['user']['location']
                favourites_count= data['retweeted_status']['user']['favourites_count']


            else :
                user_id = data['user']['id']
                name = data['user']['name']
                user_name = data['user']['screen_name']
                verified_status = data['user']['verified']
                followers_count = data['user']['followers_count']
                friends_count = data['user']['friends_count']
                statuses_count = data['user']['statuses_count']
                user_location = data['user']['location']
                favourites_count= data['user']['favourites_count']


            # insert the data into the PostgreSQL database
            store_data(user_id, name, user_name, verified_status, followers_count, friends_count, statuses_count, user_location, favourites_count)

NameError: name 'file' is not defined

In [31]:
conn = psycopg2.connect("dbname=twitter_users port=5432 user=postgres password=password")
cur = conn.cursor()


# A function that takes in a PostgreSQL query and outputs a pandas database 
def create_pandas_table(sql_query, database = conn):
    table = pd.read_sql_query(sql_query, database)
    return table

In [32]:
user_table = create_pandas_table("SELECT * FROM users_data")
user_table

,user_id,name,user_name,verified_status,followers_count,friends_count,statuses_count,user_location,favourites_count
0,375777294,TeéLaneeë🌺,TWD40_,False,637,408,18976,None,1325
1,1132273796138905600,Terri Kamp,RampTheresa,False,1522,256,20861,None,29166
...,...,...,...,...,...,...,...,...,...
15671,734263483173212160,dirty d 😽💙,deannarae21,False,684,482,43705,baerobbie 💙,75847
15672,2279976427,Venugopi,Venu7630,False,3,35,42,None,83






# III. Searching Application





Select Time Range & Save Cache

In [36]:
# set for storing keyword
stored_keyword = set()
stored_hashtag = set()
stored_date = set()
# dictionary of dictionaries
dict_text = {}
dict_hashtag = {}
dict_date = {}

# function: store dataframe in dictionary 
def put_into_cache(keyword, df, dict_set) :
    dict_set[keyword] = df.to_dict()  

In [37]:
# function: read cache from dictionary to dataframe
def dict_to_df(keyword, dict_set) :
    return pd.DataFrame.from_dict(dict_set[keyword])

In [11]:
%%time
## text search
user_keyword = input("Please enter a keyword: ")
regx = re.compile(user_keyword, re.IGNORECASE)

if (user_keyword.lower() not in stored_keyword) :
    search_tweet = pd.DataFrame(
        tweet.find({'text': {"$regex": regx}}).sort([("retweet_count", -1), ("reply_count", -1), ("favorite_count", -1)])
    )

    if (len(search_tweet) == 0) :
        print("Error: No result for %s" %user_keyword)
        search_full = None

    else :
        s_user = tuple(search_tweet["user_id"].tolist())
        params = {'l': s_user}

        cur.execute('SELECT * FROM users_data where user_id in %(l)s', params)
        search_user = pd.DataFrame(cur.fetchall())
        search_user.columns = [i[0] for i in cur.description]

        search_full = pd.merge(search_tweet, search_user, how = 'left', on = ['user_id']).drop(['_id', 'id', 'user_id'], axis = 1)
    
    stored_keyword.add(user_keyword.lower())
    put_into_cache(user_keyword.lower(), search_full, dict_text)

else :
    search_full = dict_to_df(user_keyword.lower(), dict_text)    
    
search_full

Please enter a keyword: hospital
Wall time: 809 ms


,created_at,text,retweet_count,favorite_count,reply_count,hashtags,name,user_name,verified_status,followers_count,friends_count,statuses_count,user_location,favourites_count
0,2020-04-06 20:20:33,"A young muslim man commits suicide after he is boycotted, a newborn dies after hospital refuses to accept a muslim patient, news channels talk of corona jihad by Muslims. Islamophobia taints India's response to the virus, exposing a crisis of morality. \n\nhttps://t.co/NrtdzB3sYp",6618,15733,3521,[],Rana Ayyub,RanaAyyub,True,1008827,472,29933,Mumbai,10449
1,2020-04-21 09:54:52,"Let’s respect and honour our Doctors and medical staff..\nJust convey your heartfelt gratitude to all the Doctors, Nurses,paramedical staff and other frontline workers ,who are taking care of Covid19 patients- || In recognition of her extraordinary services in treating Corona patients at South Windsor Hospital, \nDr. Uma Madhusudan, an Indian origin doctor honoured infront of her house in USA https://t.co/B5nHCLKbgN",2640,10334,67,[],Pawan Kalyan,PawanKalyan,True,3958111,161,1410,None,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
417,2020-04-25 14:47:59,"20,000 people have died in hospital from corona, this is one of those things where you start off thinking ""ahh will never happen to me"" n now it's this serious..... but still hasn't n am surprised tbh",0,0,0,[],hollie dollie,candypixies,False,871,406,22534,None,20081
418,2020-04-25 14:48:11,"Was speaking to my Nan on the phone today: “Auntie Phyllis is out of the hospital by the way” “Oh I didn’t know; what was she in for?” “That… umm… coh… coe… core… corona(!)” “Oh gosh, really? They do say it’s horrible” “Oh? Well yeah, she still sounds awful…”",0,0,0,[],Rob Gainey,RobGainey1,False,274,412,7775,"Barneveld, Nederland",25757


In [39]:
%%time
## search hashtag
user_keyword = input("Please enter a keyword: ")

if (user_keyword.lower() not in stored_hashtag) :
    # uses the index to find
    hashtag_tweet = pd.DataFrame(
        db.tweet.find({"hashtags": {"$in": [user_keyword]}}).collation( { "locale": 'en', "strength": 2 } ).sort([("retweet_count", -1), ("reply_count", -1), ("favorite_count", -1)])
    )

    if (len(hashtag_tweet) == 0) :
        print("Error: No result for %s" %user_keyword)
        hash_search_table = None

    else :
        h_user = tuple(hashtag_tweet["user_id"].tolist())
        params = {'l': h_user}

        cur.execute('SELECT * FROM users_data where user_id in %(l)s', params)
        hash_user = pd.DataFrame(cur.fetchall())
        hash_user.columns = [i[0] for i in cur.description]

        hash_search_table = pd.merge(hashtag_tweet, hash_user, how = 'left', on = ['user_id']).drop(['_id', 'id', 'user_id'], axis = 1)
    
    stored_hashtag.add(user_keyword.lower())
    put_into_cache(user_keyword.lower(), hash_search_table, dict_hashtag)

else :
    hash_search_table = dict_to_df(user_keyword.lower(), dict_hashtag)    
            
hash_search_table

Please enter a keyword: quarantine
Wall time: 820 ms


,created_at,text,retweet_count,favorite_count,reply_count,hashtags,name,user_name,verified_status,followers_count,friends_count,statuses_count,user_location,favourites_count
0,2020-04-06 23:04:04,Decluttering the House:: Winning your Never Ending Battle with Stuff do it now while your are off of work and quarantined!\nhttps://t.co/ujNiP6hL3c\n#coronaupdate #stayhome #lockdown #corona #covid19 #quarantine #quedateencasa Boris Johnson #cuomosexual Modly,179,92,6,"[coronaupdate, stayhome, lockdown, corona, covid19, quarantine, quedateencasa, cuomosexual]",Raelyn Smithe,RaelynSmithe,False,81,42,76,"Savannah, GA",11
1,2020-04-21 18:38:47,We want to make the time of the #corona #quarantine a little more pleasant. \nWe give you the chance to win @Rainbow6Game - 7560 credits.\n\nTO ENTER\nLike\nRetweet\nFollow\n@ValravneSport\nTag 2 Friends\n\nending 05.05.2020\n#WeAreVRN\nOnly GER &amp; AUT\n#StayAtHome \n#Giveaway \n#Giveaways \n#R6S https://t.co/EMKQcwx7dM,130,138,101,"[corona, quarantine, WeAreVRN, StayAtHome, Giveaway, Giveaways, R6S]",Valravn Esports,ValravnEsport,False,342,23,166,Deutschland,541
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32,2020-04-25 14:26:25,This seems #sensible\n#sarcasm #sarcasticmemes #economy #indian #corona #quarantine #funny https://t.co/XizLOsMMTG,0,0,0,"[sensible, sarcasm, sarcasticmemes, economy, indian, corona, quarantine, funny]",volatile_memers,MemersVolatile,False,1,1,11,None,2
33,2020-04-25 14:43:55,"THANK YOU @npr!!!\n\n#covid_19 #coronavirus #corona #covid19 #coronamemes #quarantine #news #breakingnews #breaking #cat #comics #stayhome #stayhomestaysafe @ Minneapolis, Minnesota https://t.co/uNYieI1v8y",0,0,0,"[covid_19, coronavirus, corona, covid19, coronamemes, quarantine, news, breakingnews, breaking]",#IStandWithIlhan,wilbeaux,False,367,670,17781,"Minneapolis, MN",12026


In [14]:
## time range search (1)
import datetime

period = True
while period :
    dateinput1 = input("Please enter the start date (format: yyyy-m-d (ex: 2020-4-13)): ")
    x1 = [int(x) for x in dateinput1.split("-")]
    dateinput2 = input("Please enter the end date (format: yyyy-m-d (ex: 2020-4-15)): ")
    x2 = [int(x) for x in dateinput2.split("-")]

    start = datetime.datetime(x1[0], x1[1], x1[2], 0, 0, 0, 0)
    end = datetime.datetime(x2[0], x2[1], x2[2], 23, 59, 59, 0)

    if (start <= end) :
        period = False
    else :
        print("Error! Try again. Start date should be earlier than end date.")

Please enter the start date (format: yyyy-m-d (ex: 2020-4-13)): 2020-4-20
Please enter the end date (format: yyyy-m-d (ex: 2020-4-15)): 2020-4-24


In [16]:
%%time
## date range search (2)
if ((dateinput1 + " to " + dateinput2) not in stored_date) :
    
    ##sort it by time (asc)
    range_tweet = pd.DataFrame(tweet.find({'created_at': {"$gte": start, "$lte": end}}).sort([("created_at", 1)]))
    
    if (len(range_tweet) == 0) :
        print("Error: No result for input range!")
        range_search_table = None
    else :
        r_user = tuple(range_tweet["user_id"].tolist())
        params = {'l': r_user}

        cur.execute('SELECT * FROM users_data where user_id in %(l)s', params)
        range_user = pd.DataFrame(cur.fetchall())
        range_user.columns = [i[0] for i in cur.description]

        range_search_table = pd.merge(range_tweet, range_user, how = 'left', on = ['user_id']).drop(['_id', 'id', 'user_id'], axis = 1)
    
    stored_date.add(dateinput1 + " to " + dateinput2)
    put_into_cache(dateinput1 + " to " + dateinput2, range_search_table, dict_date)

else :
    range_search_table = dict_to_df(dateinput1 + " to " + dateinput2, dict_date)           
     
range_search_table

Wall time: 6.98 ms


,created_at,text,retweet_count,favorite_count,reply_count,hashtags,name,user_name,verified_status,followers_count,friends_count,statuses_count,user_location,favourites_count
0,2020-04-20 01:28:36,"if u could end corona virus by sacrificing one celebrity, which one would u choose and why priyanka chopra ?",5179,21009,141,[],🔫 alveena,xxalveena,False,3364,2339,4275,london,44808
1,2020-04-20 01:50:13,Government : you can go outside now \n\nPeople’s first reaction : let’s go outside!!\n\nPeople’s reaction after Corona knacks : \n\nhttps://t.co/UV1MesxFXc,19,14,0,[],BRYAN THE MENSAH,BRYANTHEMENSAH,False,12087,2313,126944,"Accra, Ghana",55307
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1591,2020-04-24 23:57:57,"The actor told Corona DeVries, 8, “I thought this typewriter would suit you... And use it to write me back.” https://t.co/CIx6T68HCg",61,349,11,[],HuffPost,HuffPost,True,11501914,5668,557636,None,1221
1592,2020-04-24 23:59:27,"Police in India reportedly stopped lockdown regulations violators and put them in an ambulance with a faked corona virus ""patient"" https://t.co/4OLzPJu05R",135,285,70,[],𝔾𝕚𝕗𝕥 𝕄𝕦𝕣𝕒𝕡𝕒,gmurapa,False,9024,9701,47642,"Johannesburg, South Africa",107506


Implement 1

In [23]:
%%time
# using MongoDB
# most popular original tweet ranked by the retweet_count, reply_count, and favorite_count 
pop_tweet = pd.DataFrame(tweet.find({}, {'user_id': 1, 'created_at': 1, 'text': 1, 'retweet_count': 1, 'favorite_count': 1, 'reply_count': 1, 'hashtags': 1}).sort([("retweet_count", -1), ("reply_count", -1), ("favorite_count", -1)]))
pop_tweet

CPU times: user 270 ms, sys: 26.4 ms, total: 297 ms
Wall time: 1.36 s


,_id,user_id,created_at,text,retweet_count,favorite_count,reply_count,hashtags
0,5ea8cf6a82a63f13f9692cfe,1100261477989126145,2020-03-13 00:43:40,*corona virus enters my body*\n\nThe 4 Flintstone gummies I ate in 2005: https://t.co/3STfdIQtaT,237307,811062,1811,[]
1,5ea8ced182a63f13f969264d,1112592502727548928,2020-03-18 17:51:18,When this Corona shit passes we have to promise each other that we’re going to tell our kids that we survived a zombie apocalypse in 2020,181584,764405,1595,[]
...,...,...,...,...,...,...,...,...
19074,5ea8d42982a63f13f969560c,3130596629,2020-04-25 14:48:37,We have 1/4 of all deaths from #Corona - thanks to our 3rd world response by @realDonaldTrump #ashamed \n@Arpx #arleg https://t.co/ELyI69q8pp,0,0,0,"[Corona, ashamed]"
19075,5ea8d42982a63f13f969560d,2279976427,2020-04-25 14:48:38,@VSReddy_MP I think wantedly you ate damaging your leaders image @ysjagan One side there is an increase of Corona cases and also @srikakulamdistrict now attacked with corona on the other side you are giving these tweets How to understand it @ArogyaAndhra @AndhraPradeshCM @apfightscorona,0,0,0,[]


In [24]:
# TOP 10 most popular hashtag ranked by aggregating the lists of hashtags (case insensitive)
from bson.son import SON
pipeline = [
     {"$unwind": "$hashtags"},
     {"$group": {"_id": {"$toLower": "$hashtags"}, "count": {"$sum": 1}}},
     {"$sort": SON([("count", -1), ("_id", -1)])}
] 

import pprint
pprint.pprint(list(db.tweet.aggregate(pipeline))[0:10])

[{'_id': 'corona', 'count': 1248},
 {'_id': 'covid19', 'count': 542},
 {'_id': 'coronavirus', 'count': 384},
 {'_id': 'covid_19', 'count': 292},
 {'_id': 'lockdown', 'count': 166},
 {'_id': 'covid', 'count': 129},
 {'_id': 'indiafightscorona', 'count': 119},
 {'_id': 'संतरामपालजी_का_सत्संग_सुनें', 'count': 114},
 {'_id': 'stayhome', 'count': 102},
 {'_id': 'covidー19', 'count': 72}]


Implement 2

In [25]:
# using SQL
# the most active verified user ranked by statuses_count
active_user = create_pandas_table("SELECT * FROM users_data WHERE verified_status = 'True' ORDER BY statuses_count DESC")
active_user.head(5)

,user_id,name,user_name,verified_status,followers_count,friends_count,statuses_count,user_location,favourites_count
0,47596019,Liputan6.com,liputan6dotcom,True,3687560,693,1381273,Jakarta Indonesia,7062
1,16973333,The Independent,Independent,True,3202840,1146,976895,"London, England",3
2,378809160,𝔾𝕀𝔻𝕀𝕋ℝ𝔸𝔽𝔽𝕀ℂ,Gidi_Traffic,True,1676895,5826,922265,Everywhere,14478
3,2475273985,Paytm Care,Paytmcare,True,248675,15,861933,India,402
4,15448383,Inquirer,inquirerdotnet,True,3076056,1852,858348,Makati City,34260


In [26]:
# the most popular verified user ranked by followers_count
pop_user = create_pandas_table("SELECT * FROM users_data WHERE verified_status = 'True' ORDER BY followers_count DESC")
pop_user.head(5)

,user_id,name,user_name,verified_status,followers_count,friends_count,statuses_count,user_location,favourites_count
0,18839785,Narendra Modi,narendramodi,True,55786179,2364,26559,India,0
1,807095,The New York Times,nytimes,True,46361159,904,393823,New York City,18483
2,145125358,Amitabh Bachchan,SrBachchan,True,41596464,1833,66428,"Mumbai, India",75
3,101311381,Shah Rukh Khan,iamsrk,True,40026738,77,58040,None,32
4,471741741,PMO India,PMOIndia,True,34461808,486,25996,India,0


In [27]:
# the most active location ranked by tweets count
pop_location = create_pandas_table("SELECT user_location, count(*) as amount FROM users_data WHERE user_location != 'None' GROUP BY user_location ORDER BY amount DESC")
pop_location.head(5)

,user_location,amount
0,India,363
1,"New Delhi, India",185
2,United States,125
3,"Mumbai, India",121
4,"Lagos, Nigeria",112






Join the tweet table and user table

In [28]:
%%time
# using Pandas
full_table = pd.merge(tweet_table, user_table, how = 'left', on = ['user_id'])
full_table

CPU times: user 40.8 ms, sys: 17.9 ms, total: 58.8 ms
Wall time: 90.7 ms


,_id,id,user_id,created_at,text,retweet_count,favorite_count,reply_count,hashtags,name,user_name,verified_status,followers_count,friends_count,statuses_count,user_location,favourites_count
0,5ea8cd2e82a63f13f9690b8a,1253949413191344128,207809313,2020-04-25 07:30:12,India’s war with Corona is ongoing.\n\nPlay your part and make sure no one around you starves.\n\nYour contribution to defeat hunger is as valuable as defeating Corona.\n\n#FeedTheNeedy https://t.co/TUWf2nqPgZ,398,2144,108,[FeedTheNeedy],BJP,BJP4India,True,13483759,3,200328,"6-A, Deen Dayal Upadhyay Marg,",0
1,5ea8cd2e82a63f13f9690b8b,1253992905703862272,858859031464751104,2020-04-25 10:23:01,VIDEO | 25.04.20\n\n&gt; Monsta X appears on Yonhap News TV - Idol comeback through Corona... Turning on the Music industry.\n\n🔗https://t.co/kW5hsP5DeD\n\n#몬스타엑스 #MONSTAX\n#판타지아엑스 #FANTASIA_X\n@OfficialMonstaX\n\nhttps://t.co/tJN4Iyv1Gh https://t.co/GukJXWjaPW,59,144,1,"[몬스타엑스, MONSTAX, 판타지아엑스, FANTASIA_X]",Monsta X Europe,MonstaXEurope,False,27084,463,38058,let the haters bark,32020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19074,5ea8d42982a63f13f969560c,1254059747738816512,3130596629,2020-04-25 14:48:37,We have 1/4 of all deaths from #Corona - thanks to our 3rd world response by @realDonaldTrump #ashamed \n@Arpx #arleg https://t.co/ELyI69q8pp,0,0,0,"[Corona, ashamed]",Arkansas Boy,ArkansasWatch,False,276,867,4411,#Ark #LittleRock,1128
19075,5ea8d42982a63f13f969560d,1254059751379361793,2279976427,2020-04-25 14:48:38,@VSReddy_MP I think wantedly you ate damaging your leaders image @ysjagan One side there is an increase of Corona cases and also @srikakulamdistrict now attacked with corona on the other side you are giving these tweets How to understand it @ArogyaAndhra @AndhraPradeshCM @apfightscorona,0,0,0,[],Venugopi,Venu7630,False,3,35,42,None,83


Implement 3

In [29]:
%%time
# using Pandas
# search the tweets that contain the keyword 'election'
election = full_table[full_table['text'].str.contains('election',case=False)].drop(['_id', 'id', 'user_id'], axis = 1)

# rank the tweets that contain the keyword 'election' by favorite_count, retweet_count and reply_count
election.sort_values(by = ['favorite_count', 'retweet_count', 'reply_count'], ascending = (False, False, False))

CPU times: user 134 ms, sys: 8.94 ms, total: 143 ms
Wall time: 200 ms


,created_at,text,retweet_count,favorite_count,reply_count,hashtags,name,user_name,verified_status,followers_count,friends_count,statuses_count,user_location,favourites_count
4985,2020-04-06 11:31:23,"The Democrats’ all-mail ballots proposal is a ruse to legalize ballot harvesting. \n\nAnyone would be allowed to return an unlimited number of absentee ballots for voters. \n\nAt a time of so much uncertainty, we should not reduce confidence in our elections. https://t.co/hTn0MTx48m",5430,11868,1806,[],Ronna McDaniel,GOPChairwoman,True,602330,2594,14158,None,7107
4235,2020-04-25 12:51:03,Milwaukee’s health commissioner has now tied 40 coronavirus infections to the April 7 election. \n\nhttps://t.co/fGIsLKzTkm,1046,1204,76,[],Ben Wikler,benwikler,True,132408,3960,31530,"Wisconsin, USA",58028
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18955,2020-04-25 14:47:46,Every single person who was involved with this travesty should be behind bars in a covid-infested prison. || Milwaukee’s health commissioner has now tied 40 coronavirus infections to the April 7 election. \n\nhttps://t.co/fGIsLKzTkm,0,0,0,[],EmmaSofia,emmasofia3,False,162,536,16842,Gaia,4432
19071,2020-04-25 14:48:36,"40 Coronavirus Cases In Milwaukee County Linked To WI Election, yet ⁦@GovParsonMO⁩ says that voting by mail is “partisan”. The MO GOP knows their egregious agenda makes them vulnerable, so they will have us risk our health to vote them out. #MOLeg https://t.co/xpMgXHnfuy",0,0,0,[],Jess Piper,EnglishTeach07,False,5026,5369,22781,Northwest MO,94498


In [30]:
%%time
## user interactive search for text (case insensitive)
search_keyword = input()
user_search = full_table[full_table['text'].str.contains(search_keyword, case = False)].drop(['_id', 'id', 'user_id'], axis = 1)
user_search.sort_values(by = ['favorite_count', 'retweet_count', 'reply_count'], ascending = (False, False, False))

cancel
CPU times: user 105 ms, sys: 6.54 ms, total: 111 ms
Wall time: 28.5 s


,created_at,text,retweet_count,favorite_count,reply_count,hashtags,name,user_name,verified_status,followers_count,friends_count,statuses_count,user_location,favourites_count
524,2020-04-25 03:31:42,Corona? cancelled https://t.co/XHpTaCNC3t,347,4447,122,[],aksel,aksually,False,94104,605,5941,he/him,6403
6701,2020-04-24 11:43:30,"Today my beautiful daughter turns 11 🥳🧁 \nI had to cancel her birthday party due to Corona 😞\nI’m doing my best to make her birthday special regardless of the current state of events 🤘🏻\nSo, show Morgan some love in the comments and I’ll share them with her 🎉\nPls share! https://t.co/I5kUXKaJb3",137,1468,677,[],𝐒𝐡𝐚𝐧𝐧𝐨𝐧 𝐢𝐧 𝐑𝐞𝐚𝐥 𝐋𝐢𝐟𝐞 ♡,Im_shaleelee,False,4735,1287,9729,None,16781
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18399,2020-04-25 14:43:01,When you realize corona can't \ncancel your exams https://t.co/1tbhUlIt37,0,0,0,[],@meme_se_marunga,Memewala11,False,0,27,368,None,8
18979,2020-04-25 14:47:54,"@DIPRRajasthan Sir ,Best regards, due to Corona and Lok Down, Jaipur's company Traction on Demand canceled this year's placements, B.TEK students became unemployed, Sir please check this company and make efforts to get theplacements back. All students will be grateful to you",0,0,0,[],Roop Chand Jain,RoopCha58911462,False,3,83,54,None,35


Implement 4

In [31]:
%%time
# using Pandas
# search for a hashtag 'lockdown' (case insensitive)
target = ['lockdown']
lockdown = full_table[pd.DataFrame(full_table.hashtags.tolist()).applymap(lambda s:s.lower() if type(s) == str else s).isin(target).any(1)].drop(['_id', 'id', 'user_id'], axis = 1)

# rank the tweets that contain the hashtag 'lockdown' by followers_count
lockdown.sort_values(by = ['followers_count'], ascending = (False))

CPU times: user 312 ms, sys: 14.5 ms, total: 326 ms
Wall time: 416 ms


,created_at,text,retweet_count,favorite_count,reply_count,hashtags,name,user_name,verified_status,followers_count,friends_count,statuses_count,user_location,favourites_count
285,2020-04-21 13:28:03,"My wholehearted thanks&amp; heartfelt felt acknowledgment 🙏to Amudalavalasa JSP Leader Sri Pedada Rammohan garu , for distributing essentials to people who are in need. || ఆముదాలవలస నియోజకవర్గంలో లాక్ డౌన్ వేళ నిరుపేదలకు ఆహారం పంపిణీ, కరోనా పట్ల అవగాహన కార్యక్రమాలు నిర్వహిస్తున్న శ్రీ పెడాడ రామ్మోహన్ గారు.\n\n#Corona #Lockdown #JanaSeva https://t.co/1IgUA9XPBu",2020,7103,41,"[Corona, Lockdown, JanaSeva]",Pawan Kalyan,PawanKalyan,True,3958111,161,1410,None,2
2210,2020-04-25 06:17:11,Scenarios at Koyambedu market !! Social distancing at its best 😐\n\n#lockdown #CoronaUpdates #corona https://t.co/HyPC8JZwx8,16,80,10,"[lockdown, CoronaUpdates, corona]",IndiaGlitz - Tamil,igtamil,True,951444,666,68534,"Chennai, India",11776
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18963,2020-04-25 14:47:49,"Nice efforts guys..keep up the gud work✌\n#EIAForGreenIndia #covid\n#lockdown\n#Corona || This is a mechanism to legitimise some actions currently listed as violations,like projects starting construction without a valid clearance, dilutes rules by expanding the list of projects exempted from public consultation, it does not prescribe a robust system.\n#EIAForGreenIndia https://t.co/DmaY23TTvO",0,0,0,"[EIAForGreenIndia, covid, lockdown, Corona, EIAForGreenIndia]",Aarti,Aarti03524188,False,0,12,24,None,17
14087,2020-04-25 11:30:20,"How can Police officials can be so brutal, they hit a small child(wearing blue check shirt) and his hand started bleeding so much.\nThis is not acceptable #DelhiPolice \n#delhi #PoliceBrutality #childabuse #corona #lockdown https://t.co/vP9cuYR8uW",1,0,0,"[DelhiPolice, delhi, PoliceBrutality, childabuse, corona, lockdown]",Abbas Kazimi,AbbasKazimi7,False,0,46,23,"नई दिल्ली, भारत",80


In [32]:
%%time
## user interactive search for hashtag (case insensitive)

search_keyword = input()

target = [search_keyword]
lockdown = full_table[pd.DataFrame(full_table.hashtags.tolist()).applymap(lambda s:s.lower() if type(s) == str else s).isin(target).any(1)].drop(['_id', 'id', 'user_id'], axis = 1)
lockdown.sort_values(by = ['followers_count'], ascending = (False))

stayhome
CPU times: user 303 ms, sys: 10.6 ms, total: 313 ms
Wall time: 12.6 s


,created_at,text,retweet_count,favorite_count,reply_count,hashtags,name,user_name,verified_status,followers_count,friends_count,statuses_count,user_location,favourites_count
416,2020-04-25 10:21:38,"People take oath of #SocialDistancing and other measures to contain corona pandemic at Bettia, Bihar.\n\n#IndiaFightsCorona #StayHome \n\n#Report Asis Kumar https://t.co/Zp7RxAhIJo",8,50,3,"[SocialDistancing, IndiaFightsCorona, StayHome, Report]",All India Radio News,airnewsalerts,True,2359598,13,267830,"New Delhi, India",612
1092,2020-04-24 17:46:30,"Generosity is a core American value that I'm always proud to see on display in our partnerships. During this pandemic, many families are facing challenging times. Inspired to see U.S. companies pitching in to take care of their neighbors. #partners4prosperity || U.S. companies are supporting Pakistan’s efforts against corona! \n\nU.S. company @Excelerate_LP partnered with @KarachiRelief to donate a month’s supply of food and household goods to 5,000n Karachi families in need, delivered by @CareemPAK\n\n#PakistanFightsCorona #StayHome https://t.co/oOib6DBUA4",495,2180,169,"[partners4prosperity, PakistanFightsCorona, StayHome]",Secretary Pompeo,SecPompeo,True,1709561,50,2402,None,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14217,2020-04-25 14:10:32,Stay home🏘 Stay safe👨‍👩‍👧‍👦 #coronahelplineworld #covid19 #live #travel #help #stayhome #stayhomestaysafe #coronavirus #corona @ PLANET https://t.co/fjPFQOex3u https://t.co/3ERW5AfJ3o,0,0,0,"[coronahelplineworld, covid19, live, travel, help, stayhome, stayhomestaysafe]",coronahelpline.world,Corona_helpline,False,0,30,12,Planet Earth,0
17981,2020-04-25 14:40:05,@narendramodi super corona fighter #DilSeThankYou #StayHome #StayHomeStaySafe #Covid_19 So lucky to have you as our prime minister. Thank you sir https://t.co/ksShRqfeh9,0,0,0,"[DilSeThankYou, StayHome, StayHomeStaySafe, Covid_19]",Santosh Sharma,Santosh88075985,False,0,17,2,None,22


Implement 5

In [33]:
# Time
# We got the tweet data and corresponding user data in one time slot, but we save the data based on the original tweet
# try to search the earliest created original tweet that is still getting retweeted in the time slot.

In [34]:
%%time
full_table['created_at'] = pd.to_datetime(full_table['created_at'])
full_table.sort_values(by = ['created_at'], ascending = (True))

CPU times: user 12.4 ms, sys: 3.59 ms, total: 15.9 ms
Wall time: 33.3 ms


,_id,id,user_id,created_at,text,retweet_count,favorite_count,reply_count,hashtags,name,user_name,verified_status,followers_count,friends_count,statuses_count,user_location,favourites_count
14241,5ea8d18782a63f13f969432b,341578219176214528,1242575784,2013-06-03 15:32:48,Corona virus....its coming,73776,131225,9968,[],Marco,Marco_Acortes,False,5652,106,3839,None,4842
6794,5ea8cecc82a63f13f9692614,872849401928179713,189011442,2017-06-08 16:14:48,"But to return to a moment of grace.\n\n""Take me to the Seine, let us gaze into it until we become little fishes &amp; recognize each other again."" https://t.co/WfLNI92buL || ""I have often reflected that 'Corona' is your most beautiful poem""\nIB, 6 June 1949\n\nI think so, too-with Celan echoing Rilke, 'es ist Zeit.' https://t.co/CS41BlJ4jw",16,39,0,[],coldplums,coldplums,False,4341,1435,24661,on the road,41167
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19073,5ea8d42982a63f13f969560b,1254059746652389378,734263483173212160,2020-04-25 14:48:37,i will be makin tacos and gettin drunk on cinco de mayo by myself if i have to fuck a bitch named corona,0,0,0,[],dirty d 😽💙,deannarae21,False,684,482,43705,baerobbie 💙,75847
19075,5ea8d42982a63f13f969560d,1254059751379361793,2279976427,2020-04-25 14:48:38,@VSReddy_MP I think wantedly you ate damaging your leaders image @ysjagan One side there is an increase of Corona cases and also @srikakulamdistrict now attacked with corona on the other side you are giving these tweets How to understand it @ArogyaAndhra @AndhraPradeshCM @apfightscorona,0,0,0,[],Venugopi,Venu7630,False,3,35,42,None,83
